In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
from glob import glob
from os import getcwd
from os.path import join
import urllib
from collections import Counter

################################################################################
## General Stuff
################################################################################
from searchUtils import findNearest
from searchUtils import findExt
from searchUtils import findPatternExt
from fileUtils import getBasename, getDirname, getBaseFilename
from timeUtils import clock, elapsed
from webUtils import getHTML, getWebData
from timeUtils import getDateTime, isDate
from listUtils import getFlatList
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from time import sleep

################################################################################
## Database Stuff
################################################################################
from dbBase import dbBase
from mainDB import mainDB
from multiArtist import multiartist
from matchAlbums import matchAlbums
from masterdb import masterdb



from multiArtist import multiartist
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-10-09 21:04:55.469705


In [241]:
from os.path import join
from fsUtils import isFile, setFile
from ioUtils import saveFile, getFile
from pandas import DataFrame

class soundOpinions:
    def __init__(self, debug=False):
        self.debug = debug

        self.baseURL = "https://www.soundopinions.org"
        
        self.basename = "soundopinions"
        self.savedir  = "/Volumes/Piggy/Charts/data/{0}".format(self.basename)
        
        self.starterSavename = join(self.savedir, "starter.p")

        self.listsURL = "{0}/section/list".format(baseURL)
        self.listsDir = join(self.savedir, "lists")
        self.listsYearlyDir = join(self.savedir, "lists", "yearly")
        self.listsSavename = join(self.savedir, "lists.p")

        #self.showsURL = "{0}/section/list".format(baseURL)
        self.showsDir = join(self.savedir, "shows")
        self.showsSavename = join(self.savedir, "shows.p")

    def downloadURL(self, url):
        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        headers={'User-Agent':user_agent,} 

        request=urllib.request.Request(url,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read() # The data u need

        return data, response.getcode()


    def saveDownload(self, url, savename):
        if isFile(savename):
            return

        print("Downloading {0}".format(url))
        data, response = self.downloadURL(url)
        if response != 200:
            print("Error downloading {0}".format(url))
            return False
        
        print("Saving {0}".format(savename))
        saveFile(idata=data, ifile=savename)
        sleep(1)
        
        
        
    def downloadStarterURL(self):
        url      = self.baseURL
        self.saveDownload(url, self.starterSavename)
        
    def parseStarterURL(self):
        data = getHTML(self.starterSavename)
        return data
    
    
    
        
    def downloadListsURL(self):
        url      = self.listsURL
        self.saveDownload(url, self.listsSavename)
        
    def downloadYearlyLists(self):
        data = getHTML(self.listsSavename)
        aside = data.find("aside", {"class": "index"})
        if aside is not None:
            refs = aside.findAll("a")
            for ref in refs:
                name     = ref.text
                url      = "{0}{1}".format(self.baseURL, ref.attrs['href'])
                savename = join(self.listsYearlyDir, "{0}.p".format(name))
                self.saveDownload(url, savename)
                
    def downloadLists(self):
        files = findExt(self.listsYearlyDir, ext='.p')
        for ifile in files:
            data = getHTML(ifile)
            article = data.find("article", {"class": "collection"})
            if article is not None:
                uls = article.findAll("ul")
                for ul in uls:
                    for ul in uls:
                        lis = ul.findAll("li")
                        for li in lis:
                            ref  = li.find('a')
                            name = ref.text
                            url  = "{0}{1}".format(self.baseURL, ref.attrs['href'])
                            savename = setFile(self.listsDir, "{0}.p".format(name))
                            #continue
                            self.saveDownload(url, savename)
                            
                            
    def parseListLine(self, line):
        retvals = {}
        line  = line.replace("<q>", "")
        line  = line.replace("</q>", "")
        line  = line.replace(",,", ",")
        lvals = line.split(",")
        if len(lvals) > 1:        
            if lvals[1] == "000 Maniacs":
                lvals[1] = "10,000 Maniacs"
                lvals = lvals[1:]
        if len(lvals) > 5:
            if lvals[3] == " War and the Ghost of Whitey Ford":
                lvals[3] = "Love, War and the Ghost of Whitey Ford"
                lvals.pop(2)
            if lvals[3] == " Here’s the Sex Pistols":
                lvals[3] = "Never Mind the Bollocks, Here’s the Sex Pistols"
                lvals.pop(2)
        if len(lvals) > 6:        
            if lvals[3] == " Brother" and lvals[4] == " Brother":
                lvals[2] = "Brother, Brother, Brother"
                lvals.pop(3)
                lvals.pop(4)

        if len(lvals) == 5:
            retvals = dict(zip(keys, lvals))
        elif len(lvals) == 1:
            slvals = lvals[0].split(" - ")
            if len(slvals) == 2:
                skeys = ["Artist", "Title"]
                retvals = dict(zip(skeys, slvals))
            else:
                print(line)
                print(len(lvals),'\t',"\t".join(lvals))
        elif len(lvals) == 2:
            skeys = ["Artist", "Title"]
            retvals = dict(zip(skeys, lvals))
        elif len(lvals) == 3:
            skeys = ["Artist", "Title", "Year"]
            retvals = dict(zip(skeys, lvals))
        elif len(lvals) == 4:
            skeys = ["Artist", "Title", "Album", "Year"]
            retvals = dict(zip(skeys, lvals))
        elif len(lvals) == 8:
            skeys = ["Artist", "Title", "Album", "Year"]
            slvals = [lvals[0], lvals[1], lvals[2], lvals[-1]]
            retvals = dict(zip(skeys, slvals))
        else:
            return {}
            print(line)
            print(len(lvals),'\t',"\t".join(lvals))

        return retvals
    
    def getLists(self):
        files   = sorted(findExt(self.listsDir, ext='.p'))
        results = []
        
        for ifile in files:
            data = getHTML(ifile)
            section = data.find("section", {"id": "featured-songs"})
            if section is not None:
                ols = section.findAll("ol")
                for ol in ols:
                    lis = ol.findAll("li")
                    for li in lis:
                        line = li.text
                        retvals = self.parseListLine(line)
                        results.append(retvals)

        df = DataFrame(results)
        df = df[~df["Artist"].isna()]                        
        return df

In [242]:
so = soundOpinions()

In [243]:
so.downloadStarterURL()
so.downloadListsURL()
so.downloadYearlyLists()
so.downloadLists()

In [244]:
df = so.getLists()
saveFile(idata=df, ifile="SoundOpinionLists.p")

  --> This file is 357.9kB.


In [ ]:

    <main id="content" class="container">
      <aside class="index">
        <div class="player">
          <!-- @see player.js -->
        </div>
        <nav>
          <a href="/section/list/2020" class="selected">2020</a>
          <a href="/section/list/2019">2019</a>
          <a href="/section/list/2018">2018</a>
          <a href="/section/list/2017">2017</a>
          <a href="/section/list/2016">2016</a>
          <a href="/section/list/2015">2015</a>
          <a href="/section/list/2014">2014</a>
          <a href="/section/list/2013">2013</a>
          <a href="/section/list/2012">2012</a>
          <a href="/section/list/2011">2011</a>
          <a href="/section/list/2010">2010</a>
          <a href="/section/list/2009">2009</a>
          <a href="/section/list/2008">2008</a>
          <a href="/section/list/2007">2007</a>
          <a href="/section/list/2006">2006</a>
          <a href="/section/list/2005">2005</a>
        </nav>
      </aside>        

2020 	 /section/list/2020
2019 	 /section/list/2019
2018 	 /section/list/2018
2017 	 /section/list/2017
2016 	 /section/list/2016
2015 	 /section/list/2015
2014 	 /section/list/2014
2013 	 /section/list/2013
2012 	 /section/list/2012
2011 	 /section/list/2011
2010 	 /section/list/2010
2009 	 /section/list/2009
2008 	 /section/list/2008
2007 	 /section/list/2007
2006 	 /section/list/2006
2005 	 /section/list/2005


In [50]:
data

<!DOCTYPE html>
<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width" name="viewport"/>
<meta content="Sound Opinions is where people who love music can come together. On each episode, Greg Kot and Jim DeRogatis interview artists, talk about pop culture and music industry news, review new record releases and give trends a historical context. Sound Opinions is produced by WBEZ Chicago and distributed nationally by PRX, the Public Radio Exchange." name="description="/>
<link href="http://feeds.feedburner.com/TheSoundOpinionsPodcast" rel="alternate" title="Sound Opinions" type="application/rss+xml"/>
<title>Sound Opinions</title>
<link href="//static.soundopinions.org/assets/favicon.png" rel="shortcut icon"/>
<link as="style" href="/ux/veneer.css?559e2" rel="preload"/>
<link charset="utf-8" href="/ux/veneer.css?559e2" media="screen" rel="stylesheet" type="text/css"/>
<link href="